<a href="https://colab.research.google.com/github/naye971012/2023_sanhak_ML_study/blob/main/%EC%82%B0%ED%95%99%ED%98%91%EB%A0%A5_UDOP_%EC%BD%94%EB%93%9C%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#moudel import

In [ ]:
!pip install timm
!pip install transformers


In [45]:
import torch
from torch import nn
from torch import Tensor
import torch.nn.functional as F
from timm.models.vision_transformer import PatchEmbed, DropPath

In [80]:
import logging
import os
from typing import Any, Dict, Optional, Sequence, Tuple
from dataclasses import dataclass

import torch
from torch import nn
from torch import Tensor

from transformers import T5Config, T5PreTrainedModel
from transformers.modeling_outputs import BaseModelOutput
from transformers.models.t5.modeling_t5 import T5Block, T5ForConditionalGeneration, T5LayerNorm

#params

In [142]:
IN_CHAN=3 #input chanel of image, 3 since it is RGB
BATCH_SIZE=128 #batch size
IMG_SIZE=224 * 224 #image size
PATCH_SIZE=16 #patch size
TXT_SIZE = 12 #    # of text tokens
EMB_SIZE = 128 #embedding size
NUM_CLASS=10



VOCAB_SIZE = TXT_SIZE+1 #used when text embedding
MLP_LAYER = int(IMG_SIZE/pow(PATCH_SIZE,2)) + 1 + TXT_SIZE #classification decoder에 사용

#load data (구현 필요)

#embedding (현재 1D positional embedding, 추후 교체 필요)

In [127]:
class image_EmbeddingLayer(nn.Module):
    def __init__(self,in_chan, img_size, patch_size, batch_size=128):
        super().__init__()
        self.num_patches = int(img_size / pow(patch_size, 2)) # total number of patches
        self.emb_size = EMB_SIZE

        self.project = nn.Conv2d(in_chan, self.emb_size, kernel_size= patch_size, stride=patch_size) # image embedding
        self.cls_token = nn.Parameter(torch.randn(1,1,self.emb_size))
        self.positions = nn.Parameter(torch.randn(self.num_patches, self.emb_size)) # 1D positional embedding


    def forward(self, x):
        x = self.project(x)
        x = x.view(-1, self.num_patches , self.emb_size ) # [batch_size, # of patches, embedding size]
        #repeat_cls = self.cls_token.repeat(x.size()[0],1,1) #[ batch_size, 1 , 1]
        #x = torch.cat((repeat_cls, x), dim=1) #[batch_size, # of patches + 1, embedding_size]
        x += self.positions
        return x # [batch size, # of patches + 1, embedding size]

class text_EmbeddingLayer(nn.Module):
  def __init__(self,txt_size,emb_size,vocab_size):
    super().__init__()
    self.txt_size = txt_size
    self.emb_size = emb_size
    self.vocab_size = vocab_size

    self.project = nn.Embedding(num_embeddings=vocab_size,embedding_dim=self.emb_size) #num_embeddings = 단어 idx최대값 + 1
    self.cls_token = nn.Parameter(torch.randn(1,1,self.emb_size))
    self.positions = nn.Parameter(torch.randn(self.txt_size+ 1, self.emb_size)) # 1D positional embedding

  def forward(self,x):
    x = self.project(x) #[ batch size, # of token ] -> [batch size, # of token, embedding size]
    x = x.view(-1,self.txt_size,self.emb_size) # [batch_size, txt_size, embedding size]
    repeat_cls = self.cls_token.repeat(x.size()[0],1,1) #[batch_size, 1 , 1]
    x = torch.cat((repeat_cls,x),dim=1)
    x += self.positions
    return x

In [128]:
########## this block is for testing ###############
image_embedding = image_EmbeddingLayer(in_chan=IN_CHAN, img_size=IMG_SIZE, patch_size=PATCH_SIZE, batch_size = BATCH_SIZE)
text_embedding = text_EmbeddingLayer(txt_size=TXT_SIZE, emb_size=EMB_SIZE, vocab_size =VOCAB_SIZE)

image_test = torch.randn(1,3,224,224) #[batch size, # of channel, image size x, image size y] -> [batch size, # of patches, embedding size]
text_test = torch.randint(0, 11, (1, TXT_SIZE)) #[batch size, # of token] -> [batch size, # of token, embedding size]


image_output = image_embedding(image_test)
text_output = text_embedding(text_test)
print(image_output.shape, text_output.shape) # torch.Size([1, 197, 128]) torch.Size([1, 13, 128])

embedding_output = torch.cat((text_output,image_output,),dim=1) # [batch_size, # of total tokens, # of embedding layer]
print(embedding_output.shape) # torch.Size([1, 209, 128])
######### testing ends ###############################

torch.Size([1, 196, 128]) torch.Size([1, 13, 128])
torch.Size([1, 209, 128])


#encoder

In [130]:
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

In [131]:
class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [132]:
class CrossAttention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = qk_scale or head_dim ** -0.5

        self.q = nn.Linear(dim, dim, bias=qkv_bias)
        self.k = nn.Linear(dim, dim, bias=qkv_bias)
        self.v = nn.Linear(dim, dim, bias=qkv_bias)

        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x, context):
        B, N, C = x.shape
        _, N_context, _ = context.shape

        q = self.q(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        k = self.k(context).reshape(B, N_context, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        v = self.v(context).reshape(B, N_context, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [133]:
class Block(nn.Module):

    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, use_cross_attention=False):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)

        self.use_cross_attention = use_cross_attention
        if use_cross_attention:
            self.cross_attn = CrossAttention(
                dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
            self.norm_ct = norm_layer(dim)

        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x, context=None):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        if context is not None and self.use_cross_attention:
            x = x + self.drop_path(self.cross_attn(self.norm_ct(x), context))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

In [134]:
class Encoder(nn.Module):
  def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, use_cross_attention=False):
    super().__init__()
    self.block = Block(dim,num_heads, drop=drop,attn_drop=attn_drop,drop_path=drop_path)
  def forward(self,x):
    return self.block(x)

In [135]:
#########################this block is for testing ######################

encoder = Encoder(dim=EMB_SIZE, num_heads=8, drop=0., attn_drop=0., drop_path=0.)

#print(embedding_output.shape) # torch.Size([1, 209, 128]) #

encoder_output = encoder(embedding_output)

print(encoder_output.shape)

######################### testing end ##################################

torch.Size([1, 209, 128])


#decoder

In [136]:
class Decoder_classification(nn.Module):
  def __init__(self,in_feature,hidden_feature,output_feature,drop):
    super().__init__()
    self.output = Mlp(in_features=in_feature * MLP_LAYER, hidden_features=hidden_feature,out_features=output_feature, drop=0.) #final layer이므로 drop x
  def forward(self,x):
    x = x.view(-1,EMB_SIZE * MLP_LAYER)
    return self.output(x)

In [137]:
class Decoder_nlp(nn.Module):
  def __init__(self):
    super().__init__()
  def forward(self,x):
    return x

In [138]:
class Decoder(nn.Module):
  def __init__(self,in_feature,hidden_feature,output_feature,drop=0.):
    super().__init__()
    self.classification_decoder = Decoder_classification(in_feature,hidden_feature,output_feature,drop)
    self.nlp_decoder = Decoder_nlp()

  def forward(self,x,is_classification_decoder):
    if(is_classification_decoder):
      return self.classification_decoder(x)
    else:
      return self.nlp_decoder(x)

In [139]:
#########################this block is for testing ######################

decoder = Decoder(in_feature=EMB_SIZE, hidden_feature=EMB_SIZE, output_feature=NUM_CLASS, drop=0.)

#print(encoder_output.shape) # torch.Size([1, 209, 128])

decoder_output = decoder(encoder_output,is_classification_decoder=True)

print(decoder_output.shape)

######################### testing end ##################################

torch.Size([1, 10])


#model summary

In [140]:
class model_summary(nn.Module):
  def __init__(self):
    super().__init__()
    self.image_embedding = image_EmbeddingLayer(in_chan=IN_CHAN, img_size=IMG_SIZE, patch_size=PATCH_SIZE, batch_size = BATCH_SIZE)
    self.text_embedding = text_EmbeddingLayer(txt_size=TXT_SIZE, emb_size=EMB_SIZE, vocab_size =VOCAB_SIZE)
    self.encoder = Encoder(dim=EMB_SIZE, num_heads=8, drop=0., attn_drop=0., drop_path=0.)
    self.decoder = Decoder(in_feature=EMB_SIZE, hidden_feature=EMB_SIZE, output_feature=NUM_CLASS, drop=0.)

  def embedding(self,image,text):
    image_output = self.image_embedding(image)
    text_output = self.text_embedding(text)
    embedding_output = torch.cat((text_output,image_output,),dim=1) # [batch_size, # of total tokens, # of embedding layer]
    return embedding_output

  def forward(self,x):
    encoder_output = self.encoder(x)
    decoder_output = self.decoder(x,is_classification_decoder=True)
    return decoder_output

In [144]:
model = model_summary()

image_test = torch.randn(1,3,224,224) #[batch size, # of channel, image size x, image size y] -> [batch size, # of patches, embedding size]
text_test = torch.randint(0, 11, (1, TXT_SIZE)) #[batch size, # of token] -> [batch size, # of token, embedding size]

embedding_test = model.embedding(image_test,text_test)
output = model(embedding_test)

output.shape

torch.Size([1, 10])